In [5]:
import dgl
import numpy as np
import scipy.sparse as sp
import torch as th

E:\Softwares\anaconda3\envs\GML\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
data_folder = "data/"
path = data_folder + "dblp/"

In [118]:
label = np.load(path + "labels.npy")

type_num = [4057, 14328, 7723, 20]  # the number of every node type
ratio = [20, 40, 60]

# "nei_num": 1,  # the number of neighbors' types
# "n_labels": 4,
# author paper venue term

nei_p = np.load(path + "nei_p.npy", allow_pickle=True)
feat_a = sp.load_npz(path + "a_feat.npz").astype("float32")
feat_p = sp.eye(type_num[1])
apa = sp.load_npz(path + "apa.npz")
apcpa = sp.load_npz(path + "apcpa.npz")
aptpa = sp.load_npz(path + "aptpa.npz")
pos = sp.load_npz(path + "pos.npz")
train = [np.load(path + "train_" + str(i) + ".npy") for i in ratio]
test = [np.load(path + "test_" + str(i) + ".npy") for i in ratio]
val = [np.load(path + "val_" + str(i) + ".npy") for i in ratio]

label = th.LongTensor(label)

nei_p = [th.LongTensor(i) for i in nei_p]

# 源码此步preprocess_features
feat_p = th.FloatTensor(feat_p.todense())
feat_a = th.FloatTensor(feat_a.todense())
apa = th.FloatTensor(apa.todense()).to_sparse()
apcpa = th.FloatTensor(apcpa.todense()).to_sparse()
aptpa = th.FloatTensor(aptpa.todense()).to_sparse()
pos = th.FloatTensor(pos.todense()).to_sparse()

train = [th.LongTensor(i) for i in train]
val = [th.LongTensor(i) for i in val]
test = [th.LongTensor(i) for i in test]


# self.meta_paths_dict = {'APA': [('author', 'author-paper', 'paper'), ('paper', 'paper-author', 'author')],
#                                    'APTPA': [('author', 'author-paper', 'paper'), ('paper', 'paper-term', 'term'),
#                                              ('term', 'term-paper', 'paper'), ('paper', 'paper-author', 'author')],
#                                    'APVPA': [('author', 'author-paper', 'paper'), ('paper', 'paper-venue', 'venue'),
#                                              ('venue', 'venue-paper', 'paper'), ('paper', 'paper-author', 'author')],
#                                    }

In [125]:
# adj
neis = [nei_p]
links = []
for src, nei in enumerate(neis):
    dst_array_concat = th.concatenate(nei)  # .unsqueeze(0)
    src_array_concat = []
    for src_id, dst_array in enumerate(nei):
        src_array_concat.extend([src_id] * len(dst_array))
    src_array_concat = th.tensor(src_array_concat)  # .unsqueeze(0)
    index = th.vstack([src_array_concat, dst_array_concat])
    links.append(index)
    index = th.vstack([dst_array_concat, src_array_concat])
    links.append(index)

In [134]:
data_dict = {
    ("author", "author-paper", "paper"): (links[0][0], links[0][1]),
    ("paper", "paper-author", "author"): (links[1][0], links[1][1]),
}

In [142]:
feat_a.shape

torch.Size([4057, 334])

In [184]:
from openhgnn.dataset.NodeClassificationDataset import OHGB_NodeClassification,HGB_NodeClassification
# dataset = OHGB_NodeClassification("ohgbn-acm", raw_dir="./dataset", logger=None)
dataset = HGB_NodeClassification("HGBn-DBLP", raw_dir="./dataset", logger=None)


Extracting file to ./openhgnn/dataset\HGBn
Done saving data into cached files.


In [208]:
hg=dataset.g

category=dataset.category
dataset.num_classes

4

In [186]:
import numpy as np

hg.ndata['h']['paper'].shape

torch.Size([14328, 4231])

In [187]:
hg.ndata['h']['author'].shape

torch.Size([4057, 334])

In [204]:
hg.ndata['test_mask']

{'author': tensor([0, 1, 1,  ..., 1, 1, 1], dtype=torch.uint8)}

In [210]:
dataset.meta_paths_dict

{'APA': [('author', 'author-paper', 'paper'),
  ('paper', 'paper-author', 'author')],
 'APTPA': [('author', 'author-paper', 'paper'),
  ('paper', 'paper-term', 'term'),
  ('term', 'term-paper', 'paper'),
  ('paper', 'paper-author', 'author')],
 'APVPA': [('author', 'author-paper', 'paper'),
  ('paper', 'paper-venue', 'venue'),
  ('venue', 'venue-paper', 'paper'),
  ('paper', 'paper-author', 'author')]}